In [3]:
import sys
sys.path.insert(0, '../drift_inventory/utils/')
#import plot_polygon_over_raster
from plot_drift_with_polygon_overlay import 

ModuleNotFoundError: No module named 'plot_drift_with_polygon_overlay'

In [ ]:
import fiona
from shapely.geometry import shape, Polygon
import rasterio
import rasterio.plot
from rasterio.windows import Window
import matplotlib as mpl
import matplotlib.pyplot as plt
from descartes.patch import PolygonPatch
from copy import copy

In [ ]:
# get boundary box for drift from HV (happens to be watertrack)
shp = fiona.open("../gis/hv_drifts.shp", "r")
drift_poly = shp.next()
drift_bbox = shape(drift_poly['geometry']).bounds
print(drift_bbox)

In [ ]:
# we should pad these boundaries when reading in our raster so drifts don't go 
# all the way to the edge of the plot
left = int(drift_bbox[0]) - 25
bottom = int(drift_bbox[1]) - 25
right = int(drift_bbox[2]) + 25
top = int(drift_bbox[3]) + 25
height = top - bottom
width = right - left
print(left, bottom, right, top)
print(height, width)

In [ ]:
src = rasterio.open('../drift_rasters/hv_driftmask_ct_years.tif')
#rstr_shape = src.shape
rstr_bounds = src.bounds
print(src.bounds)

In [ ]:
# what is different between left polygon edge and left raster edge?
# that is how many columns to skip in our window placement
col_skip = abs(src.bounds.left - left)
col_skip

In [ ]:
# what is different between top polygon edge and top raster edge?
# that is how many rows to skip in our window placement
row_skip = src.bounds.top - top
row_skip

In [ ]:
coords = drift_poly['geometry']['coordinates'][0]
polygon = Polygon(coords)
polygon

In [ ]:
patches = [PolygonPatch(polygon, edgecolor="red", facecolor="none", linewidth=2)]

In [ ]:
win = Window(col_skip, row_skip, width, height)

with rasterio.open('../drift_rasters/hv_driftmask_ct_years.tif') as src:
    src_transform = src.transform
    win_transform = src.window_transform(win)
    w = src.read(1, window=Window(col_skip, row_skip, width, height))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    rasterio.plot.show(w, transform=win_transform, ax=ax)
    ax.add_collection(mpl.collections.PatchCollection(patches, match_original=True))


    #plt.colorbar()
    

        

In [ ]:
src_transform = src.transform
win_transform = src.window_transform(win)
w = src.read(1, window=Window(col_skip, row_skip, width, height))
fig = plt.figure()
ax = fig.add_subplot(111)
rasterio.plot.show(w, transform=win_transform, ax=ax)
ax.add_collection(mpl.collections.PatchCollection(patches, match_original=True))
